In [156]:
import pandas as pd
import math
client = "Bank of the Death Star" #change that as needed
input=r"C:\Users\CB_ll\AppData\Local\Programs\Inputs\Bank_of_the_Death_Star_Submission_11312026 - Sheet1.csv"
output=r"C:\Users\CB_ll\AppData\Local\Programs\Inputs\result_Bank_of_the_Death_Star.csv"


In [157]:
quotes = pd.read_csv(input,',')
template=pd.DataFrame(columns=["snap_date","client","instrument_type","leg2_currency","leg2_fixing","leg2_day_count","leg2_payment_frequency","option_tenor","swap_tenor","relative_strike","discount","absolute_strike","fwd_premium"])

C:\Users\CB_ll\AppData\Local\Temp\ipykernel_23608\2419970835.py:1: FutureWarning: In a future version of pandas all arguments of read_csv except for the argument 'filepath_or_buffer' will be keyword-only.
  quotes = pd.read_csv(input,',')


In [158]:
def convert_to_template (string):
    split_list = string.split()
    currency= split_list[0]
    if split_list[3]=='SONIA':
        fixing="GBP-SONIA"
        day_count="Act/365"
        payment_frequency="Annually"
        discount="GBP-SONIA"
        #TODO: use a lookup file/expand the if clause to use different fixings/daycount/payfreq if needed for EURIBOR ones
    
    #this part to extract the strike and the direction
    if (split_list[4].rfind("R")>=0):
        instrument_type="Swaption Receiver"
        if (split_list[4].find("+0")>=0):
            relative_strike="ATM"
        elif (split_list[4].find("-0")>=0):#there is no -0 in the data but just in case
            relative_strike="ATM"
        else:    
            relative_strike=-float(split_list[4].split('-')[1])
    else:
        instrument_type="Swaption Payer"
        if (split_list[4].find("+0")>=0):
            relative_strike="ATM"
        elif (split_list[4].find("-0")>=0):#there is no -0 in the data but just in case
            relative_strike="ATM"
        else:
            relative_strike=float(split_list[4].split('+')[1])
            
    #this part to extract the option tenor - checks if the first part has 2 numeric characters - if yes then we know the option tenor is of the form xx months or xx years, and if not it's x months or x years (only works for less than 100 years)
    if (split_list[2][:2].isnumeric()):
        option_tenor=split_list[2][:3]
    else:
        option_tenor=split_list[2][:2]
        
    #this part to extract the swap tenor, if the character at place -3 in the string is a letter then option tenor is only 2 chars long, otherwise 3 chars (only works for less than 100 years)
    if (split_list[2][-3].isnumeric()):
        swap_tenor=split_list[2][-3:]
    else:
        swap_tenor=split_list[2][-2:]
        
    #result of the function
    parameters_list=[instrument_type,currency,fixing,day_count,payment_frequency,option_tenor,swap_tenor,relative_strike,discount]
    return parameters_list
    
    
#convert_to_template(quotes.loc[3166].at["Trade"])

In [160]:
for index in range(len(quotes)):
    #get info from input file
    date=quotes.loc[index].at["Date"]
    premium=float(quotes.loc[index].at["Premium"])
    rate=float(quotes.loc[index].at["Forward"])
    
    #get the info from the trade data composite string ("trade" column in input file)
    parameters_list=convert_to_template(quotes.loc[index].at["Trade"])
    if parameters_list[7]=='ATM':
        relative_strike=0
    else:
        relative_strike= float(parameters_list[7])
    
    #computing absolute strike
    absolute_strike= rate+relative_strike/10000  
    
    #computing forward premium assuming there will be no tenor <1y, if there is put an if clause and split the string on "M"
    tenor=float(parameters_list[6].split('Y')[0])
    forward_premium=premium*math.exp(rate*tenor)
    
    #adding the relevant elements to the parameters list and then adding the list to the template
    parameters_list.insert(0,client)
    parameters_list.insert(0,date)
    parameters_list.append(absolute_strike)
    parameters_list.append(forward_premium)
    
    template.loc[index]=parameters_list
    
#print(template)

In [161]:
template.to_csv(output,index=False)